## Plant Sensor API Exercise
### Task

You've been given a virtual sensor API module (`sensor_api.py`) that can monitor plants. 
The API provides three functions:

- `connect(sensor_id)`: Connects to a sensor (returns True/False)
- `disconnect(sensor_id)`: Disconnects from a sensor (returns True/False)  
- `send_message(message)`: Sends a command and receives a reading (returns float or None)

#### Message Format
Messages must be formatted as: `"SENSOR_ID:COMMAND"`

Available commands:
- `SOIL_HUMIDITY`: Get soil humidity (0-100%)
- `AIR_HUMIDITY`: Get air humidity (0-100%)
- `TEMPERATURE`: Get temperature in Celsius

#### Your Task
Create a `PlantSensor` class that:

1. Stores the sensor ID when created
2. Automatically connects when initialized
3. Provides easy-to-use methods for each measurement type
4. Properly disconnects when done
5. Displays all readings in a nice format

#### Example Usage
```python
# This is how your class should work:
sensor = PlantSensor("PLANT_01")
soil = sensor.get_soil_humidity()
air = sensor.get_air_humidity()
temp = sensor.get_temperature()
sensor.display_readings()
sensor.disconnect()

### Solution

In [1]:
import sensor_api

class PlantSensor:
    def __init__(self, sensor_id):
        """Initialize the sensor with an ID and connect to it."""
        self.sensor_id = sensor_id
        self.connected = sensor_api.connect(sensor_id)
        
        # Store readings
        self.soil_humidity = None
        self.air_humidity = None
        self.temperature = None
    
    def get_soil_humidity(self):
        """Get soil humidity reading."""
        if self.connected:
            message = f"{self.sensor_id}:SOIL_HUMIDITY"
            self.soil_humidity = sensor_api.send_message(message)
            return self.soil_humidity
        return None
    
    def get_air_humidity(self):
        """Get air humidity reading."""
        if self.connected:
            message = f"{self.sensor_id}:AIR_HUMIDITY"
            self.air_humidity = sensor_api.send_message(message)
            return self.air_humidity
        return None
    
    def get_temperature(self):
        """Get temperature reading."""
        if self.connected:
            message = f"{self.sensor_id}:TEMPERATURE"
            self.temperature = sensor_api.send_message(message)
            return self.temperature
        return None
    
    def display_readings(self):
        """Display all sensor readings."""
        print(f"\n=== Plant Sensor {self.sensor_id} Readings ===")
        print(f"Soil Humidity: {self.soil_humidity}%")
        print(f"Air Humidity: {self.air_humidity}%")
        print(f"Temperature: {self.temperature}°C")
        print("=" * 35)
    
    def disconnect(self):
        """Disconnect from the sensor."""
        if self.connected:
            sensor_api.disconnect(self.sensor_id)
            self.connected = False

In [2]:
my_plant = PlantSensor("GREENHOUSE_01")

# Take readings
my_plant.get_soil_humidity()
my_plant.get_air_humidity()
my_plant.get_temperature()

# Display results
my_plant.display_readings()

# Clean up
my_plant.disconnect()

Sensor GREENHOUSE_01 connected successfully.

=== Plant Sensor GREENHOUSE_01 Readings ===
Soil Humidity: 42.8%
Air Humidity: 67.0%
Temperature: 28.8°C
Sensor GREENHOUSE_01 disconnected.


# Optional Advanced Exercises for Plant Sensor

## Foundation Setup
Before tackling the individual exercises, enhance your basic class with these improvements:

### A. Enhanced Initialization
Modify your `__init__` method to:
1. Add a `plant_name` parameter with default value `"Unknown Plant"`
2. Store readings in a dictionary structure:
   ```python
   self.readings = {
       'soil_humidity': [],
       'air_humidity': [],
       'temperature': []
   }
   ```
3. Add basic error handling for connection failures using try/except

### B. Reading Storage System
Create a private method `_get_reading(command_type)` that:
1. Checks if the sensor is connected
2. Sends the properly formatted message
3. Stores each reading with a timestamp in a dictionary:
   ```python
   reading_data = {
       'value': reading,
       'timestamp': datetime.now().strftime("%Y-%m-%d %H:%M:%S")
   }
   ```
4. Appends this dictionary to the appropriate list in `self.readings`
5. Returns the reading value

Update your existing methods (`get_soil_humidity`, etc.) to use this private method.

### C. Combined Reading Method
Add a `get_all_readings()` method that:
1. Calls all three reading methods
2. Returns a dictionary with all current readings
3. Will be extended in Exercise 2 for CSV logging

---

## Exercise 1: Water Prediction System
Create a method `predict_watering_time()` that calculates when the plant needs water.

**Requirements:**
1. Check that all reading types have at least one value
2. Get the latest value from each reading type
3. Implement this watering prediction formula:
   - Base soil humidity decrease: 2% per hour at 20°C
   - Temperature adjustment: For every degree above 20°C, add 0.3% to hourly decrease
   - Air humidity adjustment: For every 10% below 50% air humidity, add 0.2% to hourly decrease
   - Plants need water when soil humidity drops below 30%
4. Calculate hours until watering is needed
5. Return appropriate messages:
   - "Watering needed now!" if soil humidity is already ≤ 30%
   - "Watering needed in approximately X.X hours" otherwise
   - "Need all readings to predict watering time" if readings are missing

**Hint:** Use the formula:
```
hourly_decrease = base_decrease + temp_factor + humidity_factor
hours_until_watering = (current_soil - 30) / hourly_decrease
```

---

## Exercise 2: CSV Data Logging
Implement automatic logging of all sensor readings to a CSV file.

**Requirements:**
1. Modify `__init__` to accept an optional `log_file` parameter (default: None)
2. Create a private method `_initialize_csv()` that:
   - Uses the `pathlib` module to check if file exists
   - Creates a new CSV file with headers if it doesn't exist
   - Headers: timestamp, sensor_id, plant_name, soil_humidity, air_humidity, temperature
3. Create a private method `_log_to_csv(soil_hum, air_hum, temp)` that:
   - Checks if log_file is set and all values are not None
   - Appends a new row with current timestamp and all values
   - Uses the `csv` module's writer
4. Modify `get_all_readings()` to:
   - Call `_log_to_csv()` after getting all readings
   - Only log if all readings are successful (not None)

**File Structure Example:**
```csv
timestamp,sensor_id,plant_name,soil_humidity,air_humidity,temperature
2024-01-15 10:30:00,PLANT_01,Tomato Plant,45.2,55.3,22.5
```

---

## Exercise 3: Statistics Calculation
Add a method `get_statistics()` that analyzes all stored readings.

**Requirements:**
1. For each reading type that has data, calculate:
   - Minimum value
   - Maximum value  
   - Average value
   - Total count of readings
   - Timestamp of first reading
   - Timestamp of last reading
2. Use list comprehensions to extract values: `[r['value'] for r in readings_list]`
3. Calculate total readings across all types
4. Return a dictionary with all statistics
5. Handle empty reading lists appropriately

**Expected Output Structure:**
```python
{
    'soil_humidity': {
        'min': 42.3,
        'max': 48.7,
        'average': 45.5,
        'count': 5,
        'first_reading': '2024-01-15 10:00:00',
        'last_reading': '2024-01-15 10:04:00'
    },
    'total_readings': 15
}
```

---

## Exercise 4: Alert System
Create a method `check_alerts()` that monitors plant health.

**Requirements:**
1. Define threshold dictionaries for different reading types
2. Check soil humidity thresholds:
   - Critical: < 25% (immediate watering needed)
   - Warning: < 35% (water soon)
3. Check temperature thresholds:
   - Critical low: < 15°C
   - Warning low: < 18°C  
   - Warning high: > 27°C
   - Critical high: > 30°C
4. Use conditional expressions (ternary operators) for soil alerts
5. Use list comprehensions to generate temperature alerts
6. Filter out None values from alerts list
7. Return list of alert messages or ["All readings within normal range"]

**Alert Format Examples:**
- "CRITICAL: Soil humidity 22% - Water immediately!"
- "WARNING: Temperature 28°C - Getting hot"

---

## Exercise 5: Enhanced Display System
Upgrade the `display_readings()` method with advanced formatting.

**Requirements:**
1. **Header Section:**
   - Use `.center()` to create a centered header with the plant name
   - Use `.upper()` for the sensor ID
   - Create visual separators with "=" and "-" characters

2. **Readings Display:**
   - Format reading names using `.title()` and `.replace()`
   - Align values using string formatting (e.g., `{:<20}` for left align)
   - Add visual indicators based on values:
     - Soil humidity: "***" if < 30%, "*" if < 40%
     - Temperature: "!!!" if > 30°C or < 15°C, "!" if > 27°C or < 18°C

3. **Statistics Section:**
   - Only show if multiple readings exist
   - Use the statistics from Exercise 3
   - Format numbers to 1 decimal place

4. **Integration:**
   - Include watering prediction from Exercise 1
   - Show alerts from Exercise 4 with emoji prefixes:
     - 🚨 for CRITICAL alerts
     - ⚠️ for WARNING alerts

**Example Output Structure:**

```
========== Plant Monitor - Tomato Plant ==========
Sensor ID: GREENHOUSE_01
==================================================

LATEST READINGS:
----------------------------------------
  Soil Humidity         42.5%   
  Last updated:         2024-01-15 10:30:00
----------------------------------------
  Temperature           28.0°C  !
  Last updated:         2024-01-15 10:30:00
----------------------------------------

Watering needed in approximately 6.3 hours

ALERTS:
  ⚠️  WARNING: Temperature 28°C - Getting hot
==================================================
```

---

## Bonus Features

### Destructor Method
Add a `__del__` method that:
1. Checks if the sensor is still connected
2. Automatically disconnects when the object is destroyed
3. Uses `hasattr()` to safely check for the connected attribute

### Error Handling Enhancement
Improve all methods with proper try/except blocks:
1. Catch specific exceptions where possible
2. Print helpful error messages
3. Return appropriate default values



### Advanced solution

In [3]:
import sensor_api
from datetime import datetime
import csv
from pathlib import Path

class AdvancedPlantSensor:
    def __init__(self, sensor_id, plant_name="Unknown Plant", log_file=None):
        """
        Initialize the sensor with an ID and connect to it.
        
        Optional Exercise 2: Added log_file parameter for CSV logging
        """
        self.sensor_id = sensor_id
        self.plant_name = plant_name
        self.log_file = log_file
        self.readings = {
            'soil_humidity': [],
            'air_humidity': [],
            'temperature': []
        }
        
        # Try to connect
        try:
            self.connected = sensor_api.connect(sensor_id)
            if not self.connected:
                raise ConnectionError(f"Failed to connect to sensor {sensor_id}")
                
            # Optional Exercise 2: Initialize CSV file if specified
            if self.log_file:
                self._initialize_csv()
                
        except Exception as e:
            print(f"Error during initialization: {e}")
            self.connected = False
    
    def _initialize_csv(self):
        """Optional Exercise 2: Initialize CSV file with headers"""
        csv_path = Path(self.log_file)
        if not csv_path.exists():
            with open(self.log_file, 'w', newline='') as f:
                writer = csv.writer(f)
                writer.writerow(['timestamp', 'sensor_id', 'plant_name', 
                               'soil_humidity', 'air_humidity', 'temperature'])
    
    def _log_to_csv(self, soil_hum, air_hum, temp):
        """Optional Exercise 2: Log readings to CSV file"""
        if self.log_file and all(v is not None for v in [soil_hum, air_hum, temp]):
            with open(self.log_file, 'a', newline='') as f:
                writer = csv.writer(f)
                writer.writerow([
                    datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
                    self.sensor_id,
                    self.plant_name,
                    soil_hum,
                    air_hum,
                    temp
                ])
    
    def _get_reading(self, command_type):
        """Private method to get any type of reading."""
        if not self.connected:
            print("Sensor is not connected!")
            return None
        
        try:
            message = f"{self.sensor_id}:{command_type}"
            reading = sensor_api.send_message(message)
            
            if reading is not None:
                # Store reading with timestamp
                reading_data = {
                    'value': reading,
                    'timestamp': datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                }
                
                # Map command to reading type
                reading_map = {
                    'SOIL_HUMIDITY': 'soil_humidity',
                    'AIR_HUMIDITY': 'air_humidity',
                    'TEMPERATURE': 'temperature'
                }
                
                if command_type in reading_map:
                    self.readings[reading_map[command_type]].append(reading_data)
                
            return reading
            
        except Exception as e:
            print(f"Error getting reading: {e}")
            return None
    
    def get_soil_humidity(self):
        """Get soil humidity reading."""
        return self._get_reading("SOIL_HUMIDITY")
    
    def get_air_humidity(self):
        """Get air humidity reading."""
        return self._get_reading("AIR_HUMIDITY")
    
    def get_temperature(self):
        """Get temperature reading."""
        return self._get_reading("TEMPERATURE")
    
    def get_all_readings(self):
        """Get all readings at once and optionally log to CSV."""
        readings = {
            'soil_humidity': self.get_soil_humidity(),
            'air_humidity': self.get_air_humidity(),
            'temperature': self.get_temperature()
        }
        
        # Optional Exercise 2: Log to CSV if enabled
        if self.log_file:
            self._log_to_csv(
                readings['soil_humidity'],
                readings['air_humidity'],
                readings['temperature']
            )
        
        return readings
    
    def predict_watering_time(self):
        """
        Optional Exercise 1: Predict when the plant needs watering.
        
        Formula:
        - Base decrease: 2% per hour at 20°C
        - Temperature factor: +0.3% per degree above 20°C
        - Humidity factor: +0.2% per 10% below 50% air humidity
        - Watering needed when soil humidity < 30%
        """
        if not all(self.readings[key] for key in self.readings):
            return "Need all readings to predict watering time"
        
        # Get latest readings
        current_soil = self.readings['soil_humidity'][-1]['value']
        current_air = self.readings['air_humidity'][-1]['value']
        current_temp = self.readings['temperature'][-1]['value']
        
        if current_soil <= 30:
            return "Watering needed now!"
        
        # Calculate hourly decrease rate
        base_decrease = 2.0  # % per hour at 20°C
        
        # Temperature adjustment
        temp_factor = 0.3 * max(0, current_temp - 20)
        
        # Air humidity adjustment
        humidity_factor = 0.2 * max(0, (50 - current_air) / 10)
        
        # Total hourly decrease
        hourly_decrease = base_decrease + temp_factor + humidity_factor
        
        # Calculate hours until soil humidity reaches 30%
        humidity_to_lose = current_soil - 30
        hours_until_watering = humidity_to_lose / hourly_decrease
        
        return f"Watering needed in approximately {hours_until_watering:.1f} hours"
    
    def get_statistics(self):
        """
        Optional Exercise 3: Calculate statistics for all readings.
        Uses list comprehensions as taught in the notebook.
        """
        stats = {}
        
        for reading_type, readings_list in self.readings.items():
            if readings_list:
                # Using list comprehension to extract values
                values = [r['value'] for r in readings_list]
                
                stats[reading_type] = {
                    'min': min(values),
                    'max': max(values),
                    'average': sum(values) / len(values),
                    'count': len(values),
                    'first_reading': readings_list[0]['timestamp'],
                    'last_reading': readings_list[-1]['timestamp']
                }
            else:
                stats[reading_type] = {'count': 0}
        
        stats['total_readings'] = sum(
            len(readings) for readings in self.readings.values()
        )
        
        return stats
    
    def check_alerts(self):
        """
        Optional Exercise 4: Check for alerts based on thresholds.
        Returns a list of alert messages using list comprehensions.
        """
        alerts = []
        
        # Define thresholds
        thresholds = {
            'soil_humidity': {
                'critical_low': 25,
                'warning_low': 35,
                'unit': '%'
            },
            'temperature': {
                'critical_low': 15,
                'warning_low': 18,
                'warning_high': 27,
                'critical_high': 30,
                'unit': '°C'
            }
        }
        
        # Check soil humidity (using conditional expressions from notebook)
        if self.readings['soil_humidity']:
            soil = self.readings['soil_humidity'][-1]['value']
            soil_alert = (
                f"CRITICAL: Soil humidity {soil}% - Water immediately!" 
                if soil < thresholds['soil_humidity']['critical_low'] else
                f"WARNING: Soil humidity {soil}% - Water soon" 
                if soil < thresholds['soil_humidity']['warning_low'] else
                None
            )
            if soil_alert:
                alerts.append(soil_alert)
        
        # Check temperature
        if self.readings['temperature']:
            temp = self.readings['temperature'][-1]['value']
            temp_thresholds = thresholds['temperature']
            
            # Using list comprehension with conditions
            temp_alerts = [
                f"CRITICAL: Temperature {temp}°C - Too cold!" 
                if temp < temp_thresholds['critical_low'] else None,
                f"WARNING: Temperature {temp}°C - Getting cold" 
                if temp_thresholds['critical_low'] <= temp < temp_thresholds['warning_low'] else None,
                f"WARNING: Temperature {temp}°C - Getting hot" 
                if temp_thresholds['warning_high'] < temp <= temp_thresholds['critical_high'] else None,
                f"CRITICAL: Temperature {temp}°C - Too hot!" 
                if temp > temp_thresholds['critical_high'] else None
            ]
            
            # Filter out None values using list comprehension
            alerts.extend([alert for alert in temp_alerts if alert is not None])
        
        return alerts if alerts else ["All readings within normal range"]
    
    def display_readings(self):
        """
        Display all sensor readings with enhanced formatting.
        Optional Exercise 5: Uses string methods from the notebook.
        """
        # Create header with string methods
        header = f" Plant Monitor - {self.plant_name} ".center(60, '=')
        print(f"\n{header}")
        print(f"Sensor ID: {self.sensor_id.upper()}")
        print("=" * 60)
        
        # Display latest readings with formatting
        print("\nLatest Readings:".upper())
        print("-" * 40)
        
        for reading_type, readings_list in self.readings.items():
            if readings_list:
                latest = readings_list[-1]
                unit = "%" if "humidity" in reading_type else "°C"
                reading_name = reading_type.replace('_', ' ').title()
                
                # Add visual indicators based on value
                value = latest['value']
                if reading_type == 'soil_humidity':
                    indicator = "***" if value < 30 else "*" if value < 40 else ""
                elif reading_type == 'temperature':
                    indicator = "!!!" if value > 30 or value < 15 else "!" if value > 27 or value < 18 else ""
                else:
                    indicator = ""
                
                # Format the line
                line = f"  {reading_name:<20} {value:>6.1f}{unit:<3} {indicator}"
                print(line)
                print(f"  {'Last updated:':<20} {latest['timestamp']}")
                print("-" * 40)
        
        # Optional Exercise 3: Display statistics if multiple readings
        stats = self.get_statistics()
        if any(stat.get('count', 0) > 1 for stat in stats.values()):
            print("\nStatistics:".upper())
            print("-" * 40)
            for reading_type, stat in stats.items():
                if isinstance(stat, dict) and stat.get('count', 0) > 1:
                    reading_name = reading_type.replace('_', ' ').title()
                    print(f"  {reading_name}:")
                    print(f"    Range: {stat['min']:.1f} - {stat['max']:.1f}")
                    print(f"    Average: {stat['average']:.1f}")
                    print(f"    Readings: {stat['count']}")
        
        # Optional Exercise 1: Show watering prediction
        print("\n" + self.predict_watering_time())
        
        # Optional Exercise 4: Show alerts
        alerts = self.check_alerts()
        if any("CRITICAL" in alert or "WARNING" in alert for alert in alerts):
            print("\nAlerts:".upper())
            for alert in alerts:
                prefix = "  🚨 " if "CRITICAL" in alert else "  ⚠️  "
                print(prefix + alert)
        
        print("=" * 60 + "\n")
    
    def disconnect(self):
        """Disconnect from the sensor."""
        if self.connected:
            sensor_api.disconnect(self.sensor_id)
            self.connected = False
            print(f"Sensor {self.sensor_id} disconnected successfully.")
    
    def __del__(self):
        """Destructor to ensure disconnection."""
        if hasattr(self, 'connected') and self.connected:
            self.disconnect()

In [4]:
# Create sensor with CSV logging (Optional Exercise 2)
sensor = AdvancedPlantSensor("GREENHOUSE_01", "Tomato Plant", log_file="plant_log.csv")

# Take multiple readings for statistics
print("Taking readings...")
for i in range(5):
    sensor.get_all_readings()
    print(f"Reading {i+1} complete")

# Display everything
sensor.display_readings()

# Show individual optional exercises
print("\n" + "Optional Exercise Results".center(50, '-'))

# Exercise 1: Watering prediction
print(f"\n1. {sensor.predict_watering_time()}")

# Exercise 3: Statistics
print("\n3. Statistics:")
stats = sensor.get_statistics()
for key, value in stats.items():
    if isinstance(value, dict) and value.get('count', 0) > 0:
        print(f"   {key}: {value}")

# Exercise 4: Alerts
print("\n4. Current Alerts:")
for alert in sensor.check_alerts():
    print(f"   - {alert}")

# Clean up
sensor.disconnect()

# Exercise 2: Check CSV file
print(f"\n2. CSV log saved to: {sensor.log_file}")

Sensor GREENHOUSE_01 connected successfully.
Taking readings...
Reading 1 complete
Reading 2 complete
Reading 3 complete
Reading 4 complete
Reading 5 complete

=============== Plant Monitor - Tomato Plant ===============
Sensor ID: GREENHOUSE_01

LATEST READINGS:
----------------------------------------
  Soil Humidity          68.4%   
  Last updated:        2025-06-17 11:14:29
----------------------------------------
  Air Humidity           56.4%   
  Last updated:        2025-06-17 11:14:29
----------------------------------------
  Temperature            27.0°C  
  Last updated:        2025-06-17 11:14:29
----------------------------------------

STATISTICS:
----------------------------------------
  Soil Humidity:
    Range: 27.6 - 69.1
    Average: 51.2
    Readings: 5
  Air Humidity:
    Range: 31.1 - 56.4
    Average: 45.5
    Readings: 5
  Temperature:
    Range: 18.1 - 27.0
    Average: 21.8
    Readings: 5

Watering needed in approximately 9.4 hours


------------Optional E